### Bibliotecas

In [1]:
import os
import re
import shutil
import zipfile

### Parâmetros

In [7]:
# Caminhos e arquivo de entrada
input_zip_file = r'/Volumes/Dados_na_nuvem/Marcelo/Mestrado PTR/pct1_h_files.zip'
output_folder = r'/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/9_Dissertação/Processamento/Processados'

In [3]:
# Função para checar acessibilidade de um arquivo
def check_file_accessibility(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    if not os.access(file_path, os.R_OK):
        raise PermissionError(f"Erro: O arquivo '{file_path}' não possui permissão de leitura.")
    print(f"Arquivo '{file_path}' está acessível para leitura.")

# Função para checar integridade de um arquivo zip
def check_zip_integrity(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            if not zip_ref.namelist():
                raise ValueError(f"Erro: O arquivo zip '{zip_path}' está vazio.")
            print(f"O arquivo zip '{zip_path}' é válido e contém arquivos.")
    except zipfile.BadZipFile:
        raise ValueError(f"Erro: O arquivo zip '{zip_path}' está corrompido.")

# Função para checar acessibilidade do diretório de saída
def check_output_directory(output_dir):
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
            print(f"Diretório de saída '{output_dir}' foi criado com sucesso.")
        except PermissionError:
            raise PermissionError(f"Erro: Sem permissão para criar o diretório '{output_dir}'.")
    elif not os.access(output_dir, os.W_OK):
        raise PermissionError(f"Erro: Sem permissão de escrita no diretório '{output_dir}'.")
    print(f"Diretório de saída '{output_dir}' está acessível para escrita.")

In [6]:
# Executando as verificações
print("Realizando verificações de integridade...")
check_file_accessibility(input_zip_file)
check_zip_integrity(input_zip_file)
check_output_directory(output_folder)
print("Verificações concluídas. Todos os arquivos e diretórios estão em ordem.")

Realizando verificações de integridade...
Arquivo '/Volumes/Dados_na_nuvem/Marcelo/Mestrado PTR/pct1_h_files.zip' está acessível para leitura.
O arquivo zip '/Volumes/Dados_na_nuvem/Marcelo/Mestrado PTR/pct1_h_files.zip' é válido e contém arquivos.


OSError: [Errno 22] Invalid argument

In [8]:
check_output_directory(output_folder)

Diretório de saída '/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/9_Dissertação/Processamento/Processados' está acessível para escrita.


In [ ]:
import os
import re
import shutil
import zipfile

# Início do processo
print("Iniciando o processamento do arquivo zipado...")

# Extrair o número do pacote no nome do arquivo zip
print(f"Analisando o nome do arquivo zip: {os.path.basename(input_zip_file)}")
zip_basename = os.path.basename(input_zip_file)
match = re.search(r"pct(\d{1,2})_", zip_basename, re.IGNORECASE)
if not match:
    raise ValueError("Erro: O nome do arquivo zip deve conter o padrão 'pctXX_'.")

package_number = match.group(1)
raw_files_folder = os.path.join(os.path.dirname(input_zip_file), f"Pacote_{package_number}")

# Criar diretório para arquivos extraídos
print(f"Criando diretório para os arquivos extraídos: {raw_files_folder}")
os.makedirs(raw_files_folder, exist_ok=True)

# Extrair o arquivo zip para o diretório de raw files
print(f"Extraindo os arquivos do zip para o diretório: {raw_files_folder}")
with zipfile.ZipFile(input_zip_file, 'r') as zip_ref:
    zip_ref.extractall(raw_files_folder)
print("Arquivos extraídos com sucesso.")

# Criar a pasta de saída
print(f"Verificando a existência da pasta de saída: {output_folder}")
os.makedirs(output_folder, exist_ok=True)

In [10]:
raw_files_folder = '/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/9_Dissertação/Processamento/pct1_h_files' 
# Regex para identificar números em notação científica
scientific_notation_pattern = re.compile(r'-?\d+\.\d+e[+-]\d+')

# Função para converter notação científica para número inteiro
def convert_scientific_to_float(match):
    return f"{float(match.group()):.0f}"


# Processar arquivos extraídos
print("Iniciando o processamento dos arquivos '.asc'...")
for filename in os.listdir(raw_files_folder):
    if filename.endswith(".asc"):
        print(f"Processando arquivo: {filename}")
        # Verificar e converter o nome do arquivo
        new_filename = scientific_notation_pattern.sub(convert_scientific_to_float, filename)
        
        # Alterar a extensão do arquivo para .tif
        new_filename = os.path.splitext(new_filename)[0] + ".tif"
        
        # Caminhos completos do arquivo de origem e destino
        src = os.path.join(raw_files_folder, filename)
        dst = os.path.join(output_folder, new_filename)
        
        try:
            # Copiar e renomear o arquivo
            shutil.copyfile(src, dst)
            print(f"Arquivo '{filename}' copiado e renomeado para '{new_filename}'.")
        except Exception as e:
            print(f"Erro ao processar '{filename}': {e}")

print("Processamento dos arquivos '.asc' concluído.")
print("Todos os arquivos foram processados com sucesso.")

Iniciando o processamento dos arquivos '.asc'...
Processando arquivo: pct1_h_2.7648e+06.asc
Arquivo 'pct1_h_2.7648e+06.asc' copiado e renomeado para 'pct1_h_2764800.tif'.
Processando arquivo: pct1_h_2.736e+06.asc
Arquivo 'pct1_h_2.736e+06.asc' copiado e renomeado para 'pct1_h_2736000.tif'.
Processando arquivo: pct1_h_2.7396e+06.asc
Arquivo 'pct1_h_2.7396e+06.asc' copiado e renomeado para 'pct1_h_2739600.tif'.
Processando arquivo: pct1_h_2.754e+06.asc
Arquivo 'pct1_h_2.754e+06.asc' copiado e renomeado para 'pct1_h_2754000.tif'.
Processando arquivo: pct1_h_2.718e+06.asc
Arquivo 'pct1_h_2.718e+06.asc' copiado e renomeado para 'pct1_h_2718000.tif'.
Processando arquivo: pct1_h_max_2.7684e+06.asc
Arquivo 'pct1_h_max_2.7684e+06.asc' copiado e renomeado para 'pct1_h_max_2768400.tif'.
Processando arquivo: pct1_h_2.7468e+06.asc
Arquivo 'pct1_h_2.7468e+06.asc' copiado e renomeado para 'pct1_h_2746800.tif'.
Processando arquivo: pct1_h_2.6928e+06.asc
Arquivo 'pct1_h_2.6928e+06.asc' copiado e renome

In [23]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

# Criar a pasta de saída, se não existir
os.makedirs(output_folder, exist_ok=True)

# Lista de arquivos .tif
tif_files = [file for file in os.listdir(output_folder) if file.endswith('.tif')]

# Parâmetros do CRS e transformações
target_epsg_code = 31983
target_crs = CRS.from_epsg(target_epsg_code)
xmin, ymin, xmax, ymax = (313394.4779, 7343769.0758, 360591.7331, 7416126.9551)
res = 29.1591
transform = from_origin(xmin, ymax, res, res)

# Configurações para GeoTIFF
geo_tiff_config = {
    "driver": "GTiff",
    "count": 1,  # Número de bandas
    "dtype": 'float32',  # Tipo de dados
    "crs": target_crs,
    "transform": transform,
}

print("Iniciando a conversão dos arquivos .asc para GeoTIFF...")

# Loop para converter os arquivos .tif
for asc_name in tif_files:
    asc_path = os.path.join(output_folder, asc_name)
    output_name = f"{os.path.splitext(asc_name)[0]}.tif"
    output_path = os.path.join(output_folder, output_name)
    
    try:
        # Carregar os dados do arquivo .asc
        data = np.loadtxt(asc_path, skiprows=6)  # Ajuste skiprows conforme necessário
        data = np.maximum(data, 0)  # Substituir valores negativos por 0
        
        # Atualizar altura e largura no GeoTIFF config
        geo_tiff_config.update({"height": data.shape[0], "width": data.shape[1]})
        
        # Criar o GeoTIFF
        with rasterio.open(output_path, 'w', **geo_tiff_config) as dst:
            dst.write(data, 1)
        
        print(f"Arquivo convertido: {asc_name} -> {output_name}")
    
    except Exception as e:
        print(f"Erro ao processar o arquivo {asc_name}: {e}")

print("Conversão concluída com sucesso.")

Iniciando a conversão dos arquivos .asc para GeoTIFF...
Arquivo convertido: pct1_h_2692800.tif -> pct1_h_2692800.tif
Arquivo convertido: pct1_h_2768400.tif -> pct1_h_2768400.tif
Arquivo convertido: pct1_h_2710800.tif -> pct1_h_2710800.tif
Arquivo convertido: pct1_h_2728800.tif -> pct1_h_2728800.tif
Arquivo convertido: pct1_h_2750400.tif -> pct1_h_2750400.tif
Arquivo convertido: pct1_h_2721600.tif -> pct1_h_2721600.tif
Arquivo convertido: pct1_h_2707200.tif -> pct1_h_2707200.tif
Arquivo convertido: pct1_h_2703600.tif -> pct1_h_2703600.tif
Arquivo convertido: pct1_h_2725200.tif -> pct1_h_2725200.tif
Arquivo convertido: pct1_h_2739600.tif -> pct1_h_2739600.tif
Arquivo convertido: pct1_h_2754000.tif -> pct1_h_2754000.tif
Arquivo convertido: pct1_h_2714400.tif -> pct1_h_2714400.tif
Arquivo convertido: pct1_h_2743200.tif -> pct1_h_2743200.tif
Arquivo convertido: pct1_h_2696400.tif -> pct1_h_2696400.tif
Arquivo convertido: pct1_h_2718000.tif -> pct1_h_2718000.tif
Arquivo convertido: pct1_h_27

In [19]:
import os
import geopandas as gpd

# Caminho do arquivo
path = '/Users/marcelofernandes/PycharmProjects/SP_urban_flooding/bounding_box_bacia.json'

# Verificar se o arquivo existe
if os.path.exists(path):
    print("Arquivo encontrado!")
    if os.access(path, os.R_OK):
        try:
            # Tentar carregar com GeoPandas
            bounding_box_baseline = gpd.read_file(path)
            print("Bounding box carregado com sucesso!")
        except Exception as e:
            print(f"Erro ao carregar o bounding box com GeoPandas: {e}")
    else:
        print("Arquivo encontrado, mas sem permissão de leitura!")
else:
    print("Arquivo não encontrado!")


Arquivo encontrado!
Bounding box carregado com sucesso!


In [21]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# Diretórios e configurações
clipped_folder = os.path.join(output_folder, "clipped_files")  # Diretório para salvar os arquivos clipados
os.makedirs(clipped_folder, exist_ok=True)

# Converter geometria do bounding box para o formato esperado pelo rasterio
bounding_box_geometry = [geom for geom in bounding_box_baseline.geometry]

# Processar arquivos .tif
print("Iniciando a clipagem dos arquivos .tif...")
for filename in os.listdir(output_folder):
    if filename.endswith(".tif"):
        print(f"Processando arquivo: {filename}")
        src_path = os.path.join(output_folder, filename)
        
        try:
            # Abrir o arquivo raster
            with rasterio.open(src_path) as src:
                # Checar CRS
                if bounding_box_baseline.crs != src.crs:
                    bounding_box_baseline = bounding_box_baseline.to_crs(src.crs)
                
                # Verificar sobreposição
                raster_bounds = box(*src.bounds)
                bounding_box_union = bounding_box_baseline.unary_union
                if not raster_bounds.intersects(bounding_box_union):
                    print(f"A geometria do bounding box não se sobrepõe ao raster: {filename}")
                    continue
                
                # Realizar a clipagem
                out_image, out_transform = mask(src, [bounding_box_union], crop=True)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                    "crs": src.crs
                })
                
                # Salvar o arquivo clipado
                clipped_filename = os.path.splitext(filename)[0] + "_clp_ARCDV.tif"
                output_path = os.path.join(clipped_folder, clipped_filename)
                with rasterio.open(output_path, "w", **out_meta) as dest:
                    dest.write(out_image)
                
                # Definir o caminho de saída com o novo nome
                clipped_filename = os.path.splitext(filename)[0] + "_clp_ARCDV.tif"
                output_path = os.path.join(clipped_folder, clipped_filename)
                
                # Salvar o arquivo clipado
                with rasterio.open(output_path, "w", **out_meta) as dest:
                    dest.write(out_image)
                    
                print(f"Arquivo clipado salvo em: {output_path}")
        except Exception as e:
            print(f"Erro ao processar '{filename}': {e}")

print("Clipagem concluída para todos os arquivos .tif.")

Iniciando a clipagem dos arquivos .tif...
Processando arquivo: pct1_h_2692800.tif
Erro ao processar 'pct1_h_2692800.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2768400.tif
Erro ao processar 'pct1_h_2768400.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2710800.tif
Erro ao processar 'pct1_h_2710800.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2728800.tif
Erro ao processar 'pct1_h_2728800.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2750400.tif
Erro ao processar 'pct1_h_2750400.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2721600.tif
Erro ao processar 'pct1_h_2721600.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2707200.tif
Erro ao processar 'pct1_h_2707200.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2703600.tif
Erro ao processar 'pct1_h_2703600.tif': Must pass either crs or epsg.
Processando arquivo: pct1_h_2725200.tif
Erro ao processar 'pct1_h_2725200.tif'